# Prompt Management with Vertex AI SDK

------------------------------------
Detailed Jupyter notebook tutorial on managing prompts using Vertex AI
Includes: creating, versioning, listing, modifying, inference, deletion, and tagging


## Setup


In [ ]:
!pip install --upgrade google-cloud-aiplatform --quiet

In [ ]:
import os
from google.cloud import aiplatform
from vertexai import Client, types
from google import genai
from google.genai import types as genai_types


In [ ]:
PROJECT = "your-gcp-project-id"
LOCATION = "us-central1"
CLIENT = Client(project=PROJECT, location=LOCATION)
print(f"Vertex AI client created for project {PROJECT}, location {LOCATION}")


## Register the prompt to Vertex AI



In [ ]:
## Define a local prompt

define_prompt = types.Prompt(
    prompt_data=types.PromptData(
        contents=[
            genai_types.Content(parts=[genai_types.Part(text="Hello, {name}! How are you doing today?")])
        ],
        variables=[{"name": genai_types.Part(text="Alice")}, {"name": genai_types.Part(text="Bob")}],
        model="models/text-generation-model-name",
    )
)
print("Local prompt defined.")



In [ ]:
registered_prompt = CLIENT.prompts.create(prompt=define_prompt)
print("Prompt registered with ID:", registered_prompt.prompt_id)


## Modify and create a new version


In [ ]:
modified_prompt = types.Prompt(
    prompt_data=types.PromptData(
        contents=[
            genai_types.Content(parts=[genai_types.Part(text="Hi, {name}! Welcome back. How may I help you today?")])
        ],
        variables=[{"name": genai_types.Part(text="Charlie")}, {"name": genai_types.Part(text="Dana")}],
        model="models/text-generation-model-name",
    )
)
new_version = CLIENT.prompts.create_version(prompt=modified_prompt, prompt_id=registered_prompt.prompt_id)
print("New prompt version created:", new_version.version_id)


## Retrieve a prompt and its version, run inference

In [ ]:
retrieved_prompt = CLIENT.prompts.get(prompt_id=registered_prompt.prompt_id)
print("Retrieved prompt ID:", retrieved_prompt.prompt_id)

retrieved_version = CLIENT.prompts.get_version(
    prompt_id=registered_prompt.prompt_id, version_id=new_version.version_id
)
print("Retrieved prompt version:", retrieved_version.version_id)


In [ ]:
# List prompts and versions

print("Listing prompts in project:")
for p in CLIENT.prompts.list():
    print(" •", p.prompt_id)

print(f"Listing versions for prompt {registered_prompt.prompt_id}:")
for v in CLIENT.prompts.list_versions(prompt_id=registered_prompt.prompt_id):
    print(" • Version ID:", v.version_id)


In [ ]:
#  Run inference with a loaded prompt
genai_client = genai.Client(vertexai=True, project=PROJECT, location=LOCATION)
contents = retrieved_version.prompt_data.contents
assembled_prompt = retrieved_version.assemble_contents()
response = genai_client.models.generate_content(
    model=retrieved_version.prompt_data.model, contents=assembled_prompt
)
print("Generated response:", response)


In [ ]:
# Label or alias a prompt version
alias_prompt = CLIENT.prompts.create_version(
    prompt=types.Prompt(
        prompt_data=types.PromptData(
            contents=[
                genai_types.Content(parts=[genai_types.Part(text="Production: Hello {name}, how may I assist you today?")])
            ],
            variables=[{"name": genai_types.Part(text="Eve")}, {"name": genai_types.Part(text="Frank")}],
            model="models/text-generation-model-name",
        ),
    ),
    prompt_id=registered_prompt.prompt_id,
    config=types.CreatePromptConfig(
        display_name="my-prompt-prod",
        labels={"environment": "production"},
    ),
)
print("Alias version created:", alias_prompt.version_id, alias_prompt.labels)


## Delete a prompt version and entire prompt


In [ ]:
CLIENT.prompts.delete_version(
    prompt_id=registered_prompt.prompt_id, version_id=new_version.version_id
)
print(f"Deleted version {new_version.version_id}")

CLIENT.prompts.delete(prompt_id=registered_prompt.prompt_id)
print(f"Deleted prompt {registered_prompt.prompt_id}")


In [ ]:
# Restore a previous version
restored = CLIENT.prompts.restore_version(
    prompt_id=registered_prompt.prompt_id, version_id="your-version-id-to-restore"
)
print("Restored version ID:", restored.version_id)

